In [1]:
import pandas as pd
data_info = pd.read_csv('../data.info')

In [4]:
data_info['label'].shape

(121838,)

In [3]:
unique_ids = pd.Series(data_info['gene_id'].unique())
len(unique_ids)

3852

In [4]:
train_gene_id = unique_ids.sample(int(len(unique_ids)*0.8), random_state = 4266)

In [5]:
data_info_train = data_info[data_info['gene_id'].isin(train_gene_id)]
data_info_valid = data_info[~data_info['gene_id'].isin(train_gene_id)]
data_info_train.shape[0]+data_info_valid.shape[0]==data_info.shape[0]

True

In [6]:
data_info_valid['label'].value_counts()/len(data_info_valid)

0    0.956529
1    0.043471
Name: label, dtype: float64

In [7]:
import json
jsonlist = []
with open('../dataset0.json') as f:
    for jsonobj in f:
        jsonlist.append(json.loads(jsonobj))

In [8]:
from tqdm import tqdm

In [21]:
import numpy as np
df_list = []
for i in tqdm(jsonlist):
    transcript_id = list(i.keys())[0]
    transcript_position = list(i[transcript_id].keys())[0]
    five_mer = list(i[transcript_id][transcript_position].keys())[0]
    readings = i[transcript_id][transcript_position][five_mer]
    df = pd.DataFrame(readings)
    df.columns = ['dwell_time_-1','sd_-1','mean_-1','dwell_time_0','sd_0','mean_0','dwell_time_1','sd_1','mean_1']
    df['product_mean_dwell_-1'] = df['dwell_time_-1']*df['mean_-1']
    df['product_mean_dwell_0'] = df['dwell_time_0']*df['mean_0']
    df['product_mean_dwell_1'] = df['dwell_time_1']*df['mean_1']
    df['product_var_dwell_-1'] = df['sd_-1']*df['sd_-1']*(df['dwell_time_-1'])
    df['product_var_dwell_0'] = df['sd_0']*df['sd_0']*(df['dwell_time_0'])
    df['product_var_dwell_1'] = df['sd_1']*df['sd_1']*(df['dwell_time_1'])

    weighted_mean_neg1 = df['product_mean_dwell_-1'].sum()/df['dwell_time_-1'].sum()
    weighted_mean_0 = df['product_mean_dwell_0'].sum()/df['dwell_time_0'].sum()
    weighted_mean_1 = df['product_mean_dwell_1'].sum()/df['dwell_time_1'].sum()

    weighted_sd_neg1 = np.sqrt(df['product_var_dwell_-1'].sum()/df['dwell_time_-1'].sum())
    weighted_sd_0 = np.sqrt(df['product_var_dwell_0'].sum()/df['dwell_time_0'].sum())
    weighted_sd_1 = np.sqrt(df['product_var_dwell_1'].sum()/df['dwell_time_1'].sum())
    
    mean_25_neg1 = df['mean_-1'].quantile(0.25)
    mean_25_0 = df['mean_0'].quantile(0.25)
    mean_25_1 = df['mean_1'].quantile(0.25)

    mean_50_neg1 = df['mean_-1'].quantile(0.5)
    mean_50_0 = df['mean_0'].quantile(0.5)
    mean_50_1 = df['mean_1'].quantile(0.5)

    mean_75_neg1 = df['mean_-1'].quantile(0.75)
    mean_75_0 = df['mean_0'].quantile(0.75)
    mean_75_1 = df['mean_1'].quantile(0.75)
    
    df_list.append([transcript_id,transcript_position,five_mer,weighted_mean_neg1,weighted_mean_0,weighted_mean_1,weighted_sd_neg1,
                    weighted_sd_0,weighted_sd_1,mean_25_neg1,mean_25_0,mean_25_1,mean_50_neg1,mean_50_0,mean_50_1,mean_75_neg1,
                    mean_75_0,mean_75_1])
    """
    df_list.append([transcript_id,transcript_position,five_mer,weighted_mean_neg1,weighted_mean_0,weighted_mean_1,weighted_sd_neg1,
                    weighted_sd_0,weighted_sd_1])
    """



  0%|          | 0/121838 [00:00<?, ?it/s]

100%|██████████| 121838/121838 [19:18<00:00, 105.16it/s]


In [23]:
new_df = pd.DataFrame(df_list)

In [25]:
new_df.columns = ['transcript_id','transcript_position','five_mer','weighted_mean_neg1','weighted_mean_0','weighted_mean_1','weighted_sd_neg1',
                    'weighted_sd_0','weighted_sd_1','mean_25_neg1','mean_25_0','mean_25_1','mean_50_neg1','mean_50_0','mean_50_1','mean_75_neg1',
                    'mean_75_0','mean_75_1']

In [26]:
new_df

,transcript_id,transcript_position,five_mer,weighted_mean_neg1,weighted_mean_0,weighted_mean_1,weighted_sd_neg1,weighted_sd_0,weighted_sd_1,mean_25_neg1,mean_25_0,mean_25_1,mean_50_neg1,mean_50_0,mean_50_1,mean_75_neg1,mean_75_0,mean_75_1
0,ENST00000000233,244,AAGACCA,123.762870,125.793483,80.775369,5.209304,8.511144,5.686085,123.0,124.00,79.000,125.0,126.0,80.50,126.0,128.0,82.000
1,ENST00000000233,261,CAAACTG,109.924484,108.101783,94.108586,3.910096,3.735377,3.442957,108.0,106.00,92.800,110.0,108.0,94.10,112.0,110.0,95.725
2,ENST00000000233,316,GAAACAG,105.450998,99.426169,89.309704,3.400684,3.910397,2.501662,105.0,97.90,88.400,106.0,99.5,89.20,106.0,101.0,90.100
3,ENST00000000233,332,AGAACAT,129.548782,97.842815,89.096953,6.542432,3.132313,2.372598,128.0,96.60,86.975,130.0,97.5,89.90,131.0,98.9,91.225
4,ENST00000000233,368,AGGACAA,118.217577,121.925694,84.996204,7.490865,6.224289,4.516338,116.0,121.00,83.625,118.0,122.0,85.40,119.0,123.0,86.900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121833,ENST00000641834,1348,GGGACAT,117.736860,116.644041,81.604529,3.679951,5.318444,4.342628,117.0,114.00,79.800,118.0,116.0,82.10,119.0,119.0,83.900
121834,ENST00000641834,1429,CTGACAC,111.417384,115.248207,80.183728,5.835194,9.597531,3.848341,110.0,113.00,78.900,112.0,116.0,80.60,113.0,117.0,81.900
121835,ENST00000641834,1531,TGGACAC,113.627173,113.706528,84.184182,4.313860,5.075660,2.339143,113.0,112.75,83.425,114.0,114.0,84.50,115.0,115.0,85.400
121836,ENST00000641834,1537,CTGACCA,109.836890,123.712610,82.354740,3.270741,7.163837,2.847985,108.0,121.00,80.500,110.0,124.0,82.00,112.0,125.0,83.800


In [27]:
new_df.columns

Index(['transcript_id', 'transcript_position', 'five_mer',
       'weighted_mean_neg1', 'weighted_mean_0', 'weighted_mean_1',
       'weighted_sd_neg1', 'weighted_sd_0', 'weighted_sd_1', 'mean_25_neg1',
       'mean_25_0', 'mean_25_1', 'mean_50_neg1', 'mean_50_0', 'mean_50_1',
       'mean_75_neg1', 'mean_75_0', 'mean_75_1'],
      dtype='object')

In [30]:
new_df['5-mer-0'] = new_df['five_mer'].map(lambda x:x[0])
new_df['5-mer-1'] = new_df['five_mer'].map(lambda x:x[1])
new_df['5-mer-2'] = new_df['five_mer'].map(lambda x:x[2])
new_df['5-mer-3'] = new_df['five_mer'].map(lambda x:x[3])
new_df['5-mer-4'] = new_df['five_mer'].map(lambda x:x[4])
new_df['5-mer-5'] = new_df['five_mer'].map(lambda x:x[5])
new_df['5-mer-6'] = new_df['five_mer'].map(lambda x:x[6])

In [31]:
new_df['5-mer-4'].value_counts()

C    121838
Name: 5-mer-4, dtype: int64

In [32]:
new_df=new_df.drop(columns = ['5-mer-3','5-mer-4'])

In [38]:
new_df

,transcript_id,transcript_position,five_mer,weighted_mean_neg1,weighted_mean_0,weighted_mean_1,weighted_sd_neg1,weighted_sd_0,weighted_sd_1,mean_25_neg1,...,mean_75_0,mean_75_1,5-mer-0,5-mer-1,5-mer-2,5-mer-5,5-mer-6,5-mer_window-1,5-mer_window0,5-mer_window1
0,ENST00000000233,244,AAGACCA,123.762870,125.793483,80.775369,5.209304,8.511144,5.686085,123.0,...,128.0,82.000,A,A,G,C,A,AAGAC,AGACC,GACCA
1,ENST00000000233,261,CAAACTG,109.924484,108.101783,94.108586,3.910096,3.735377,3.442957,108.0,...,110.0,95.725,C,A,A,T,G,CAAAC,AAACT,AACTG
2,ENST00000000233,316,GAAACAG,105.450998,99.426169,89.309704,3.400684,3.910397,2.501662,105.0,...,101.0,90.100,G,A,A,A,G,GAAAC,AAACA,AACAG
3,ENST00000000233,332,AGAACAT,129.548782,97.842815,89.096953,6.542432,3.132313,2.372598,128.0,...,98.9,91.225,A,G,A,A,T,AGAAC,GAACA,AACAT
4,ENST00000000233,368,AGGACAA,118.217577,121.925694,84.996204,7.490865,6.224289,4.516338,116.0,...,123.0,86.900,A,G,G,A,A,AGGAC,GGACA,GACAA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121833,ENST00000641834,1348,GGGACAT,117.736860,116.644041,81.604529,3.679951,5.318444,4.342628,117.0,...,119.0,83.900,G,G,G,A,T,GGGAC,GGACA,GACAT
121834,ENST00000641834,1429,CTGACAC,111.417384,115.248207,80.183728,5.835194,9.597531,3.848341,110.0,...,117.0,81.900,C,T,G,A,C,CTGAC,TGACA,GACAC
121835,ENST00000641834,1531,TGGACAC,113.627173,113.706528,84.184182,4.313860,5.075660,2.339143,113.0,...,115.0,85.400,T,G,G,A,C,TGGAC,GGACA,GACAC
121836,ENST00000641834,1537,CTGACCA,109.836890,123.712610,82.354740,3.270741,7.163837,2.847985,108.0,...,125.0,83.800,C,T,G,C,A,CTGAC,TGACC,GACCA


In [36]:
new_df['5-mer_window-1'] = new_df['five_mer'].map(lambda x: x[0:5])
new_df['5-mer_window0'] = new_df['five_mer'].map(lambda x: x[1:6])
new_df['5-mer_window1'] = new_df['five_mer'].map(lambda x: x[2:7])

In [37]:
new_df

,transcript_id,transcript_position,five_mer,weighted_mean_neg1,weighted_mean_0,weighted_mean_1,weighted_sd_neg1,weighted_sd_0,weighted_sd_1,mean_25_neg1,...,mean_75_0,mean_75_1,5-mer-0,5-mer-1,5-mer-2,5-mer-5,5-mer-6,5-mer_window-1,5-mer_window0,5-mer_window1
0,ENST00000000233,244,AAGACCA,123.762870,125.793483,80.775369,5.209304,8.511144,5.686085,123.0,...,128.0,82.000,A,A,G,C,A,AAGAC,AGACC,GACCA
1,ENST00000000233,261,CAAACTG,109.924484,108.101783,94.108586,3.910096,3.735377,3.442957,108.0,...,110.0,95.725,C,A,A,T,G,CAAAC,AAACT,AACTG
2,ENST00000000233,316,GAAACAG,105.450998,99.426169,89.309704,3.400684,3.910397,2.501662,105.0,...,101.0,90.100,G,A,A,A,G,GAAAC,AAACA,AACAG
3,ENST00000000233,332,AGAACAT,129.548782,97.842815,89.096953,6.542432,3.132313,2.372598,128.0,...,98.9,91.225,A,G,A,A,T,AGAAC,GAACA,AACAT
4,ENST00000000233,368,AGGACAA,118.217577,121.925694,84.996204,7.490865,6.224289,4.516338,116.0,...,123.0,86.900,A,G,G,A,A,AGGAC,GGACA,GACAA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121833,ENST00000641834,1348,GGGACAT,117.736860,116.644041,81.604529,3.679951,5.318444,4.342628,117.0,...,119.0,83.900,G,G,G,A,T,GGGAC,GGACA,GACAT
121834,ENST00000641834,1429,CTGACAC,111.417384,115.248207,80.183728,5.835194,9.597531,3.848341,110.0,...,117.0,81.900,C,T,G,A,C,CTGAC,TGACA,GACAC
121835,ENST00000641834,1531,TGGACAC,113.627173,113.706528,84.184182,4.313860,5.075660,2.339143,113.0,...,115.0,85.400,T,G,G,A,C,TGGAC,GGACA,GACAC
121836,ENST00000641834,1537,CTGACCA,109.836890,123.712610,82.354740,3.270741,7.163837,2.847985,108.0,...,125.0,83.800,C,T,G,C,A,CTGAC,TGACC,GACCA


In [41]:
new_df.columns

Index(['transcript_id', 'transcript_position', 'five_mer',
       'weighted_mean_neg1', 'weighted_mean_0', 'weighted_mean_1',
       'weighted_sd_neg1', 'weighted_sd_0', 'weighted_sd_1', 'mean_25_neg1',
       'mean_25_0', 'mean_25_1', 'mean_50_neg1', 'mean_50_0', 'mean_50_1',
       'mean_75_neg1', 'mean_75_0', 'mean_75_1', '5-mer-0', '5-mer-1',
       '5-mer-2', '5-mer-5', '5-mer-6', '5-mer_window-1', '5-mer_window0',
       '5-mer_window1'],
      dtype='object')

In [42]:
new_df_cat = new_df[['5-mer-0','5-mer-1','5-mer-2','5-mer-5','5-mer-6','5-mer_window-1', '5-mer_window0', '5-mer_window1']]

In [43]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder()
one_hot_encoded = encoder.fit_transform(new_df_cat)

In [44]:
one_hot_encoded_df = pd.DataFrame(one_hot_encoded.toarray(), columns=encoder.get_feature_names_out(input_features=new_df_cat.columns))

In [45]:
one_hot_encoded_df

,5-mer-0_A,5-mer-0_C,5-mer-0_G,5-mer-0_T,5-mer-1_A,5-mer-1_G,5-mer-1_T,5-mer-2_A,5-mer-2_G,5-mer-5_A,...,5-mer_window1_GACAG,5-mer_window1_GACAT,5-mer_window1_GACCA,5-mer_window1_GACCC,5-mer_window1_GACCG,5-mer_window1_GACCT,5-mer_window1_GACTA,5-mer_window1_GACTC,5-mer_window1_GACTG,5-mer_window1_GACTT
0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121833,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
121834,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
121835,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
121836,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
data_encoded = pd.concat([new_df.drop(columns=new_df_cat.columns), one_hot_encoded_df], axis=1)

In [48]:
data_encoded['A_freq'] = data_encoded['five_mer'].map(lambda x:x.count('A'))
data_encoded['C_freq'] = data_encoded['five_mer'].map(lambda x:x.count('C'))
data_encoded['G_freq'] = data_encoded['five_mer'].map(lambda x:x.count('G'))
data_encoded['T_freq'] = data_encoded['five_mer'].map(lambda x:x.count('T'))

In [49]:
data_encoded

,transcript_id,transcript_position,five_mer,weighted_mean_neg1,weighted_mean_0,weighted_mean_1,weighted_sd_neg1,weighted_sd_0,weighted_sd_1,mean_25_neg1,...,5-mer_window1_GACCG,5-mer_window1_GACCT,5-mer_window1_GACTA,5-mer_window1_GACTC,5-mer_window1_GACTG,5-mer_window1_GACTT,A_freq,C_freq,G_freq,T_freq
0,ENST00000000233,244,AAGACCA,123.762870,125.793483,80.775369,5.209304,8.511144,5.686085,123.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4,2,1,0
1,ENST00000000233,261,CAAACTG,109.924484,108.101783,94.108586,3.910096,3.735377,3.442957,108.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3,2,1,1
2,ENST00000000233,316,GAAACAG,105.450998,99.426169,89.309704,3.400684,3.910397,2.501662,105.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4,1,2,0
3,ENST00000000233,332,AGAACAT,129.548782,97.842815,89.096953,6.542432,3.132313,2.372598,128.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4,1,1,1
4,ENST00000000233,368,AGGACAA,118.217577,121.925694,84.996204,7.490865,6.224289,4.516338,116.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4,1,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121833,ENST00000641834,1348,GGGACAT,117.736860,116.644041,81.604529,3.679951,5.318444,4.342628,117.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2,1,3,1
121834,ENST00000641834,1429,CTGACAC,111.417384,115.248207,80.183728,5.835194,9.597531,3.848341,110.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2,3,1,1
121835,ENST00000641834,1531,TGGACAC,113.627173,113.706528,84.184182,4.313860,5.075660,2.339143,113.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2,2,2,1
121836,ENST00000641834,1537,CTGACCA,109.836890,123.712610,82.354740,3.270741,7.163837,2.847985,108.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2,3,1,1


In [50]:
df_train = data_encoded[data_encoded['transcript_id'].isin(data_info_train['transcript_id'])]
df_valid = data_encoded[~data_encoded['transcript_id'].isin(data_info_train['transcript_id'])]

In [51]:
df_train['transcript_position'] = df_train['transcript_position'].astype(int)
df_train = pd.merge(df_train, data_info_train, on = ['transcript_id','transcript_position'])

C:\Users\xghan\AppData\Local\Temp\ipykernel_2036\3048243875.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['transcript_position'] = df_train['transcript_position'].astype(int)


In [52]:
df_valid['transcript_position'] = df_valid['transcript_position'].astype(int)
df_valid = pd.merge(df_valid, data_info_valid,on=["transcript_id","transcript_position"])

C:\Users\xghan\AppData\Local\Temp\ipykernel_2036\286692896.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_valid['transcript_position'] = df_valid['transcript_position'].astype(int)


In [53]:
df_valid.shape[0]+df_train.shape[0] == new_df.shape[0]

True

In [54]:
df_valid.to_csv('label_df_valid.csv', index = False)
df_train.to_csv('label_df_train.csv', index = False)